# Decision Trees

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, confusion_matrix, RocCurveDisplay, roc_curve
from sklearn.datasets import load_iris

%matplotlib inline

# Objectives

- Describe the decision tree modeling algorithm
- Use attribute selection methods to build different trees
- Explain the pros and cons of decision trees
- Interpret the feature importances of a fitted model

# Decision Trees at a High Level

> **Decision trees** are a supervised learning model that uses past data to form a graph/pathway which leads to the model making _decisions_ on its predictions.

I like to think of decision trees as a bunch of forks in the road.

<a title="Jonathan Billinger / Fork in the road" href="https://commons.wikimedia.org/wiki/File:Fork_in_the_road_-_geograph.org.uk_-_1355424.jpg"><img width="512" alt="Fork in the road - geograph.org.uk - 1355424" src="https://upload.wikimedia.org/wikipedia/commons/7/71/Fork_in_the_road_-_geograph.org.uk_-_1355424.jpg"></a>

Every time we make a decision, we split up, or *partition*, the data based on the features.

## Simple Example of a Decision Tree

Let's say we have this set of data:

Work Status |  Age  | Favorite Website
------------|-------|-------------------------
 Student    | Young | A
 Working    | Young | B
 Working    | Old   | C
 Working    | Young | B
 Student    | Young | A
 Student    | Young | A



This can help us answer a couple questions:

- If someone is a young worker, what website do we recommend?
- If someone is an old worker, what website then?

### Picturing Decisions as a Tree

![](images/simple_decision_tree.png)

> Note our tree would look different depending on where we made our decisions.

# Important Terminology Related to Decision Trees

- **Root Node:** Represents entire population or sample.
- **Decision Node:** Node that is split.
- **Leaf/ Terminal Node:** Node with no children.
- **Pruning:** Removing nodes.
- **Branch / Sub-Tree:** A sub-section of a decision tree.
- **Parent and Child Node:** A node divided into sub-nodes is the parent; the sub-nodes are its children.

<img src='./images/decision_leaf.webp' width=600 />

## Overview of Algorithm's Steps

> Here's a great visual of a decision tree  http://www.r2d3.us/visual-intro-to-machine-learning-part-1/

1. Organize data features and target
2. Make a *decision* (a split) based on some *metric* using the features
    * Data are split into partitions via *branches*
    
    
3. Continue on with each partition, and do more splits for each using the features in that partition
4. Keep doing that until a **stopping condition** is hit
    * Number of data points in a final partition
    * Layers deep
    
    
5. To make predictions, run data point/s through the decision nodes (the forks in the road)

Now we have to determine what metric we use to make our split/decision!

# Entropy/Information Gain and Gini

The goal is to have our ultimate classes be fully "ordered" (for a binary dependent variable, we'd have the 1's in one group and the 0's in the other).

![](images/information_gain_split.png)

One way to assess the value of a split is to measure how *disordered* our groups are, and there is a notion of *entropy* that measures precisely this.

## Entropy

The entropy of the whole dataset is given by:

$\large E = -\Sigma^n_i p_i\log_2(p_i)$,

where $p_i$ is the probability of belonging to the $i$th group, where $n$ is the number of groups (i.e. target values).

**Entropy will always be between 0 and 1. The closer to 1, the more disordered your group.**

### Information Gain

To calculate the entropy of a *split*, we're going to want to calculate the entropy of each of the groups made by the split, and then calculate a weighted average of those groups' entropies––weighted, that is, by the size of the groups.

For a given split, the **information gain** is simply the entropy of the parent group less the entropy of the split.

> We can essentially say **information gain** is the **_difference_** of the **parent's entropy** and the **_average_** of the **children's entropy**

For a given parent, then, we maximize our model's performance by *minimizing* the split's entropy.

What we'd like to do then is:

1. Look at the entropies of all possible splits
2. Choose the split with the lowest entropy

In practice there are far too many splits for it to be practical for a person to calculate all these different entropies ...

... but we can make computers do these calculations for us!

Moreover, we can **iterate** this algorithm on the resultant groups until we reach pure groups!

**Question**: Are we in fact guaranteed, proceeding in this way, to reach pure groups, no matter what our data looks like?

**Observation**: This algorithm looks for the best split **locally**. There is no regard for how an overall tree might look. That's what makes this algorithm ***greedy***.

## Gini Impurity

An alternative metric to entropy comes from the work of Corrado Gini. The Gini Impurity is defined as:

$\large G = 1 - \Sigma_ip_i^2$, or, equivalently, $\large G = \Sigma_ip_i(1-p_i)$.

where, again, $p_i$ is the probability of belonging to the $i$th group.

**Gini Impurity will always be between 0 and 0.5. The closer to 0.5, the more disordered your group.**

1. Compute gini impurity for all possible splits
2. Would choose the 'best' split aka the one that has the lowest gini score (is the most pure)
3. Gini score added up for each child node

# With `sklearn`

## Setting up Data

In [ ]:
iris = load_iris() 
print(iris.target_names)
print(iris.feature_names)

In [ ]:
# petal length and width features
feature_used = iris.feature_names[2:]
X = iris.data[:, 2:] 
y = iris.target 

X

In [ ]:
y

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

## Training the Model

> Check out the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) documentation

In [ ]:
tree_clf_nodepth = None
tree_clf_depth = None

In [ ]:
tree_clf_nodepth.fit(X_train, y_train)
tree_clf_depth.fit(X_train, y_train)

## Predictions and Evaluation

In [ ]:
y_pred = tree_clf_nodepth.predict(X_test)
y_pred

In [ ]:
y_pred_depth = tree_clf_depth.predict(X_test)
y_pred_depth

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred_depth)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, tree_clf_nodepth.predict(X_test))).plot()

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, tree_clf_depth.predict(X_test))).plot()

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
plot_tree(tree_clf_nodepth, ax=ax);

In [ ]:
feature_used

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
plot_tree(tree_clf_depth, ax=ax);

# Challenges with Decision Trees

## Decision Trees are Prone to Overfitting

In [ ]:
# Using more data features this time
feature_used = iris.feature_names[:]
X = iris.data[:, :]
y = iris.target 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

X_train.shape, X_test.shape

In [ ]:
# Allow it to run the full default hyperparameters
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

In [ ]:
# Accuracy on training data
tree_clf.score(X_train, y_train)

In [ ]:
# Accuracy on test data
tree_clf.score(X_test, y_test)

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
plot_tree(tree_clf, ax=ax);

## Bias-Variance with Decision Trees

The CART algorithm will repeatedly partition data into smaller and smaller subsets until those final subsets are homogeneous in terms of the outcome variable. In practice this often means that the final subsets (known as the leaves of the tree) each consist of only one or a few data points. 

This tends to result in low-bias, high variance models.

### Stopping Criterion - Pruning Parameters

The recursive binary splitting procedure described above needs to know when to stop splitting as it works its way down the tree with the training data.

**min_samples_leaf:**  The most common stopping procedure is to use a minimum count on the number of training instances assigned to each leaf node. If the count is less than some minimum then the split is not accepted and the node is taken as a final leaf node.

**max_leaf_nodes:** 
Reduce the number of leaf nodes.

**max_depth:**
Reduce the depth of the tree to build a generalized tree.

**min_impurity_decrease:**
A node will split if the impurity decrease in the split is above the threshold, otherwise it will be a leaf.

In [ ]:
# Stop it from running too long
tree_clf_tuned = DecisionTreeClassifier(min_samples_leaf=2, random_state=42)
tree_clf_tuned.fit(X_train, y_train)
# Accuracy on training data & test data
print('Training:', tree_clf_tuned.score(X_train, y_train))
print('Testing:', tree_clf_tuned.score(X_test, y_test))

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
plot_tree(tree_clf_tuned, ax=ax);

# Feature Importances

The fitted tree has an attribute called `ct.feature_importances_`. What does this mean? Roughly, the importance (or "Gini importance") of a feature is a sort of weighted average of the impurity decrease at internal nodes that make use of the feature. The weighting comes from the number of samples that depend on the relevant nodes.

> The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. It is also known as the Gini importance. See [`sklearn`'s documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.feature_importances_).

In [ ]:
tree_clf_tuned.feature_importances_

In [ ]:
for importance, feature in zip(tree_clf_tuned.feature_importances_, feature_used):
    print(importance, feature)

In [ ]:
dict(zip(tree_clf_tuned.feature_importances_, feature_used))

More on feature importances [here](https://towardsdatascience.com/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3).

# Conclusions

- The decision tree is a "white-box" type of ML algorithm. It shares internal decision-making logic, which is not available in the black-box type of algorithms such as Neural Network.
- Its training time is faster compared to other algorithms such as neural networks.
- The decision tree is a non-parametric method, which does not depend upon probability distribution assumptions.
- Decision trees can handle high-dimensional data with good accuracy.

## Pros

- Easy to interpret and visualize
- Can easily capture non-linear patterns
- Require little data preprocessing from the user (no need to normalize data)
- Can be used for feature engineering such as variable selection and predicting missing values 
- Make no assumptions about distribution because its non-parametric

## Cons

- Sensitive to noisy data (overfit)
- Trouble with imbalanced datasets